In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#!zip -F file.zip --out file-large.zip
#!unzip file-large.zip
#!unzip "/content/drive/My Drive/img_align_celeba.zip"
!cat '/content/drive/My Drive/utils.py'
!cat '/content/drive/My Drive/ops.py'
!cat '/content/drive/My Drive/models_64x64.py'
import sys
import imageio
sys.path.append('/content/drive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import re
import scipy
import numpy as np
import tensorflow as tf
import cv2
import imageio

from collections import OrderedDict


def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        path_dir, _ = os.path.split(path)
        if not os.path.isdir(path_dir):
            os.makedi

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models

def to_range(images, min_value=0.0, max_value=1.0, dtype=None):
    """
    transform images from [-1.0, 1.0] to [min_value, max_value] of dtype
    """
    assert \
        np.min(images) >= -1.0 - 1e-5 and np.max(images) <= 1.0 + 1e-5 \
        and (images.dtype == np.float32 or images.dtype == np.float64), \
        'The input images should be float64(32) and in the range of [-1.0, 1.0]!'
    if dtype is None:
        dtype = images.dtype
    return ((images + 1.) / 2. * (max_value - min_value) + min_value).astype(dtype)


def batchimwrite(image, path, cnt, prob):
    """ save an [-1.0, 1.0] image """

    for i in range(image.shape[0]):
        image = np.array(image, copy=True)

        if (prob[i] > -4.5 and cnt < 50) :
            imageio.imwrite("%s_%d.jpg" % (path, cnt) , to_range(image[i, :, :, :], 0, 255, np.uint8))
            cnt += 1
            print(cnt,prob[i])

    return cnt

""" param """
epoch = 50
batch_size = 100
lr = 0.0002
z_dim = 100
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]
tf.reset_default_graph()

""" graphs """
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator

    ''' graph '''
    # inputs
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, training=False ,reuse=False)
    f_logit = discriminator(fake,training=False ,reuse=False)


""" train """
''' init '''
# session
sess = utils.session()
# saver
saver = tf.train.Saver()

''' initialization '''
ckpt_dir = '/content/drive/My Drive/GP/checkpoints/celeba_dcgan/Epoch_(5)_(2175of3165).ckpt'
saver.restore(sess, ckpt_dir)
sess.run(tf.global_variables_initializer())
saver.restore(sess, ckpt_dir)

try:
    
    cnt = 0
    while(True):
        if(cnt >= 50):
          break
        z_ipt = np.random.normal(size=[batch_size, z_dim])
        img = sess.run(fake, feed_dict={z: z_ipt})
        prob = sess.run(f_logit,{z: z_ipt})
        loss = tf.reduce_mean(prob)
        l = sess.run(loss)
        print(l)
        save_dir = '/content/drive/My Drive/results_dcgan/test'
        utils.mkdir(save_dir + '/')
        cnt = batchimwrite(img, '%s/img' % (save_dir), cnt, prob)


except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/GP/checkpoints/celeba_dcgan/Epoch_(5)_(2175of3165).ckpt
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/GP/checkpoints/celeba_dcgan/Epoch_(5)_(2175of3165).ckpt
-3.0695233
1 [-2.963564]
2 [-2.4600554]
3 [-1.850536]
4 [-2.5242686]
5 [-3.701902]
6 [-2.6039734]
7 [-2.862306]
8 [-2.517914]
9 [-2.6923482]
10 [-0.9518598]
11 [-3.3290508]
12 [-3.0636315]
13 [-3.1607516]
14 [-3.7339392]
15 [-2.7652266]
16 [-2.5166419]
17 [-2.9956656]
18 [-2.9109795]
19 [-4.334607]
20 [-3.218203]
21 [-2.3239996]
22 [-3.643551]
23 [-3.3223248]
24 [-3.1568365]
25 [-1.2529931]
26 [-3.218162]
27 [-2.9451156]
28 [-3.5184782]
29 [-1.2141782]
30 [-3.5605958]
31 [-3.2917185]
32 [-4.431612]
33 [-1.5870966]
34 [-3.3159237]
35 [-2.3520682]
36 [-1.1997216]
37 [-2.6637683]
38 [-2.660379]
39 [0.00089039]
40 [-2.1964862]
41 [-2.7465408]
42 [-2.3536758]
43 [-2.6455069]
44 [-1.1448296]
45 [-1.9529983]
46 [-2.9788349]
47 [-4.3652334]
48 [-3.384990

In [ ]:
!zip -r '/content/drive/My Drive/results/images7.zip' '/content/drive/My Drive/results/celeba_dcgan7'